In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import sys

sys.path.append('/usr/local/cuda/lib64')
sys.path.append('/usr/local/lib/python2.7/dist-packages/')
sys.path.append('/usr/local/lib/python2.7/site-packages/')

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sys
!conda install --yes --prefix {sys.prefix} xlrd

Fetching package metadata .......
Solving package specifications: ..........

Package plan for installation in environment /root/miniconda3/envs/carnd-term1:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlrd-1.1.0                 |           py35_0         169 KB

The following NEW packages will be INSTALLED:

    xlrd: 1.1.0-py35_0

Fetching packages ...
xlrd-1.1.0-py3 100% |################################| Time: 0:00:00  19.11 MB/s
Extracting packages ...
[      COMPLETE      ]|###################################################| 100%
Linking packages ...
[      COMPLETE      ]|###################################################| 100%


In [5]:

# coding: utf-8

# In[1]:

import pandas as pd
import os 
import cv2
import numpy as np


# In[2]:

demographics = pd.read_excel('/src/Data/ANIML/oasis_cross-sectional.xls', sheetname=3) # load data
df = demographics.dropna(how='any') # remove NaN values
df_columns = list(demographics.columns)
X_columns = np.delete(df_columns, [6,7], None) # X matrix won't have MMSE or CDR scores
Xdf = df.reindex(columns=X_columns)
X = Xdf.values # creating X


# In[3]:

# Define function to get list of pngs based on slice number
pngs_path='/src/Data/ANIML/OASIS_pngs'
def getaxialPNG(path):
    l = []
    axialslice90_files = []
    axialslice91_files = []
    axialslice92_files = []
    axialslice93_files = []
    axialslice94_files = []
    axialslice95_files = []
    axialslice96_files = []
    axialslice97_files = []
    axialslice98_files = []
    axialslice99_files = []
    
    for root, directories, filenames in os.walk(path):
        
        for filename in filenames:
            if ".90." in filename: 
                axialslice90_files.append(os.path.join(root, filename))
            if ".91." in filename: 
                axialslice91_files.append(os.path.join(root, filename))
            if ".92." in filename: 
                axialslice92_files.append(os.path.join(root, filename))
            if ".93." in filename: 
                axialslice93_files.append(os.path.join(root, filename))
            if ".94." in filename: 
                axialslice94_files.append(os.path.join(root, filename))
            if ".95." in filename: 
                axialslice95_files.append(os.path.join(root, filename))
            if ".96." in filename: 
                axialslice96_files.append(os.path.join(root, filename))
            if ".97." in filename: 
                axialslice97_files.append(os.path.join(root, filename))
            if ".98." in filename: 
                axialslice98_files.append(os.path.join(root, filename))
            if ".99." in filename: 
                axialslice99_files.append(os.path.join(root, filename))

    l = list(zip(axialslice90_files, axialslice91_files, axialslice92_files, axialslice93_files, axialslice94_files, axialslice95_files, axialslice96_files, axialslice97_files, axialslice98_files, axialslice99_files))

    return ((np.asarray(l)))

axial_files0 = getaxialPNG(pngs_path)
axial_X_files = np.take(axial_files0, indices=df.index.values, axis=0) # keeps the images with the same index as X matrix
axial90loc, axial91loc, axial92loc, axial93loc, axial94loc, axial95loc, axial96loc, axial97loc, axial98loc, axial99loc = zip(*axial_X_files)


# In[4]:

X_id, X_sex, X_handedness, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF = zip(*X) # unzips big X matrix

def sex_translator(X_sex):
    X_sex_binary = []
    X_sex_encoded = []
    for x in X_sex:
        if x == 'M':
            X_sex_binary.append(1)
            X_sex_encoded.append([0,1])
        else:
            X_sex_binary.append(-1)
            X_sex_encoded.append([1,0])
    
    return(zip(X_sex_binary, X_sex_encoded)) # gives us binary and one-hot encoded for sex
           
def hand_translator(X_handedness):
    X_hand_binary = []
    X_hand_encoded = []
    for x in X_handedness:
        if x == 'R':
            X_hand_binary.append(1)
            X_hand_encoded.append([0,1])
        else:
            X_hand_binary.append(-1)
            X_hand_encoded.append([1,0])
    
    return(zip(X_hand_binary, X_hand_encoded)) # same as above but for handedness

# turns out all the patients are right-handed
# maybe we should all just be left-handed so we don't suffer from Alzheimer's
# /frequentist sarcasm

X_sex_binary, X_sex_encoded = zip(*sex_translator(X_sex)) # unzipping to get our function outputs
X_hand_binary, X_hand_encoded = zip(*hand_translator(X_handedness))


# In[5]:

def prepPNGimgs(array_of_image_paths):
    l = []
    for img_file in array_of_image_paths: # for each file in the list of images...
        img = cv2.imread("{}".format(img_file)) # read the image...
        img = cv2.resize(np.array(img), (224,224)) # resize it to 224 by 224 QUICK WHAT'S 224 SQUARED??
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # makes it grayscale
        flatten = gray.flatten() # spaghettifies it into 1 by 50176 (which is 224 squared)
        l.append(flatten)
    
    return(np.asarray(l))


# In[6]:

Y_CDR_columns = [column_name for column_name in df_columns if column_name == 'CDR']
Y_CDR_df = df.reindex(columns=Y_CDR_columns)
Y_CDR = Y_CDR_df.values

Y_MMSE_columns = [column_name for column_name in df_columns if column_name == 'MMSE']
Y_MMSE_df = df.reindex(columns=Y_MMSE_columns)
Y_MMSE = Y_MMSE_df.values

CDR_threshold_0 = 0 # threshold values by CDR scale
CDR_threshold_0point5 = 0.5
CDR_threshold_1 = 1

MMSE_threshold_24 = 24 # threshold values by MMSE scale
MMSE_threshold_18 = 18

def CDR_probable_AD_thresholder(Y_CDR, threshold_value):
    Y_CDR_binary = []
    Y_CDR_encoded = []
    for y in Y_CDR:
        if y > threshold_value:
            Y_CDR_binary.append(1)
            Y_CDR_encoded.append([0,1])
        else:
            Y_CDR_binary.append(-1)
            Y_CDR_encoded.append([1,0])

    return((zip(Y_CDR_binary, Y_CDR_encoded)))

def MMSE_probable_Dementia_thresholder(Y_MMSE, threshold_value):
    Y_MMSE_binary = []
    Y_MMSE_encoded = []
    for y in Y_MMSE:
        if y < threshold_value:
            Y_MMSE_binary.append(1)
            Y_MMSE_encoded.append([0,1])
        else:
            Y_MMSE_binary.append(-1)
            Y_MMSE_encoded.append([1,0])
        
    return(zip(Y_MMSE_binary, Y_MMSE_encoded))

Y_CDR_binary, Y_CDR_encoded = zip(*CDR_probable_AD_thresholder(Y_CDR, CDR_threshold_0))
Y_MMSE_binary, Y_MMSE_encoded = zip(*MMSE_probable_Dementia_thresholder(Y_MMSE, MMSE_threshold_24))


# In[7]:

# turning everything into numpy arrays because why not

df_index = np.asarray(df.index.values)
X_id = np.asarray(X_id)
X_sex = np.asarray(X_sex)
X_sex_binary = np.asarray(X_sex_binary)
X_sex_encoded = np.asarray(X_sex_encoded)
X_handedness = np.asarray(X_handedness)
X_hand_binary = np.asarray(X_hand_binary)
X_hand_encoded = np.asarray(X_hand_encoded)
X_age = np.asarray(X_age) 
X_education = np.asarray(X_education) 
X_SES = np.asarray(X_SES)
X_eTIV = np.asarray(X_eTIV)
X_nWBV = np.asarray(X_nWBV)
X_ASF = np.asarray(X_ASF)
axial90loc = np.asarray(axial90loc) 
axial91loc = np.asarray(axial91loc)
axial92loc = np.asarray(axial92loc) 
axial93loc = np.asarray(axial93loc)
axial94loc = np.asarray(axial94loc)
axial95loc = np.asarray(axial95loc) 
axial96loc = np.asarray(axial96loc) 
axial97loc = np.asarray(axial97loc) 
axial98loc = np.asarray(axial98loc) 
axial99loc = np.asarray(axial99loc)
axial90_spaghetti = prepPNGimgs(axial90loc)
axial91_spaghetti = prepPNGimgs(axial91loc)
axial92_spaghetti = prepPNGimgs(axial92loc)
axial93_spaghetti = prepPNGimgs(axial93loc)
axial94_spaghetti = prepPNGimgs(axial94loc)
axial95_spaghetti = prepPNGimgs(axial95loc)
axial96_spaghetti = prepPNGimgs(axial96loc)
axial97_spaghetti = prepPNGimgs(axial97loc)
axial98_spaghetti = prepPNGimgs(axial98loc)
axial99_spaghetti = prepPNGimgs(axial99loc)
Y_CDR = np.squeeze(np.asarray(Y_CDR))
Y_CDR_binary = np.asarray(Y_CDR_binary)
Y_CDR_encoded = np.asarray(Y_CDR_encoded)
Y_MMSE = np.squeeze(np.asarray(Y_MMSE))
Y_MMSE_binary = np.asarray(Y_MMSE_binary)
Y_MMSE_encoded = np.asarray(Y_MMSE_encoded)


# In[9]:

FinalX = np.vstack((X_id, X_sex, X_sex_binary, X_handedness, X_hand_binary, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF))
FinalX = FinalX.T

FinalY = np.vstack((Y_CDR, Y_CDR_binary, Y_MMSE, Y_MMSE_binary))
FinalY = FinalY.T

In [7]:
# concatenating and then turning them into dataframes then into CSVs

XDATAFRAME = pd.DataFrame((FinalX), index=df_index, columns=['X_id', 'X_sex', 'X_sex_binary', 'X_handedness', 'X_hand_binary', 'X_age', 'X_education', 'X_SES', 'X_eTIV', 'X_nWBV', 'X_ASF'])

YDATAFRAME = pd.DataFrame((FinalY), index=df_index, columns=['Y_CDR', 'Y_CDR_binary', 'Y_MMSE', 'Y_MMSE_binary'])

classifiers = pd.DataFrame((Y_CDR_encoded), index=df_index, columns=['NC','AD'])

# NeuralNetDF = pd.concat([axial90s, ouput], axis=1)

# XDATAFRAME.to_csv('Data/ANIML/X_DataFrame.csv')
# YDATAFRAME.to_csv('Data/ANIML/Y_DataFrame.csv')
# NeuralNetDF.to_csv('Data/ANIML/NN_Data_frame_90.csv') 

In [8]:
axial90s = pd.concat([pd.DataFrame((axial90_spaghetti), index=df_index), classifiers], axis=1)
axial91s = pd.concat([pd.DataFrame((axial91_spaghetti), index=df_index), classifiers], axis=1)
axial92s = pd.concat([pd.DataFrame((axial92_spaghetti), index=df_index), classifiers], axis=1)
axial93s = pd.concat([pd.DataFrame((axial93_spaghetti), index=df_index), classifiers], axis=1)
axial94s = pd.concat([pd.DataFrame((axial94_spaghetti), index=df_index), classifiers], axis=1)
axial95s = pd.concat([pd.DataFrame((axial95_spaghetti), index=df_index), classifiers], axis=1)
axial96s = pd.concat([pd.DataFrame((axial96_spaghetti), index=df_index), classifiers], axis=1)
axial97s = pd.concat([pd.DataFrame((axial97_spaghetti), index=df_index), classifiers], axis=1)
axial98s = pd.concat([pd.DataFrame((axial98_spaghetti), index=df_index), classifiers], axis=1)
axial99s = pd.concat([pd.DataFrame((axial99_spaghetti), index=df_index), classifiers], axis=1)

# axial91s = pd.DataFrame((axial91_spaghetti), index=df_index)
# axial92s = pd.DataFrame((axial92_spaghetti), index=df_index)
# axial93s = pd.DataFrame((axial93_spaghetti), index=df_index)
# axial94s = pd.DataFrame((axial94_spaghetti), index=df_index)
# axial95s = pd.DataFrame((axial95_spaghetti), index=df_index)
# axial96s = pd.DataFrame((axial96_spaghetti), index=df_index)
# axial97s = pd.DataFrame((axial97_spaghetti), index=df_index)
# axial98s = pd.DataFrame((axial98_spaghetti), index=df_index)
# axial99s = pd.DataFrame((axial99_spaghetti), index=df_index)

allSlices = pd.concat([axial90s, axial91s, axial93s, axial94s, axial95s, axial96s, axial97s, axial98s, axial99s],axis=0)

#shuffle rows
allSlicesShuff = allSlices.sample(frac=1, random_state=1).reset_index(drop=True)

X_train0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[0:1000].as_matrix()
y_train0 = allSlicesShuff['AD'][0:1000].as_matrix()

X_test0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[1000:].as_matrix()
y_test0 = allSlicesShuff['AD'][1000:].as_matrix()


In [9]:
X_train, y_train, X_test, y_test = X_train0, y_train0, X_test0, y_test0

In [12]:
class LeNetMnistClassifier():
    @staticmethod
    def toDigit(hot_encode):
        return np.argmax(hot_encode)
        
    def __init__(self, **kwargs):
        if kwargs.get("model_path", None):
            from keras.models import load_model
            self.model = load_model(kwargs['model_path'])
        else:
            self.activation_function = kwargs.pop('activation_function', 'relu')
            self.batch_size = kwargs.pop('batch_size', 128)
            self.epochs = kwargs.pop('epochs', 5)
            self.kernal_size = kwargs.pop('kernal_size', (3, 3))
            self.loss_function = kwargs.pop('loss_function', 'categorical_crossentropy')
            self.optimizer = kwargs.pop('optimizer', 'sgd')
            self.pool_size = kwargs.pop('pool_size', (2, 2))
            self.model = self._model()
            
    
    def _model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function,
                 input_shape=(224,224,1)))
        # One additional convolutional layer (32 channels)
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function))
        model.add(Conv2D(32, kernel_size=self.kernal_size, padding='same',
                 activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        # One additional convolutional layer (64 channels)
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        model.add(Conv2D(64, self.kernal_size, padding='same', activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512, activation=self.activation_function))
        model.add(Dropout(0.25))
        model.add(Dense(2, activation='softmax'))
        
        return model
    
    def predict(self, digits=()):
        return self.model.predict(digits)
    
    def evaluate(self, X_test=None, y_test=None):
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)
        X_test = X_test.astype('float32')
        X_test/=255        
        number_of_classes = 2
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        return self.model.evaluate(X_test, y_test)
    
    def preprocess_and_train(self, X_train=None, y_train=None, X_test=None, y_test=None):
        self._train(*self._preprocess(X_train, y_train, X_test, y_test))
        
    def _preprocess(self, X_train, y_train, X_test, y_test):
        X_train = X_train.reshape(X_train.shape[0], 224, 224, 1)
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        
        X_train/=255
        X_test/=255
        
        number_of_classes = 2
        y_train = np_utils.to_categorical(y_train, number_of_classes)
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        
        return X_train, y_train, X_test, y_test
    
    def _train(self, X_train, y_train, X_test, y_test):
        self.model.compile(loss=self.loss_function,
              optimizer=self.optimizer,
              metrics=['accuracy'])
        
        fit_output = self.model.fit(X_train,
                        y_train,
                        batch_size=self.batch_size,
                        epochs=self.epochs,
                        verbose=1,
                        validation_data=(X_test, y_test))
        self._history = fit_output.history
        
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        self.model.save("oasis_test_{0}.h5".format(timestr)) 
    
    @property
    def history(self):
        return self._history

In [13]:
classifier = LeNetMnistClassifier(epochs=30, optimizer=keras.optimizers.Adadelta())
classifier.preprocess_and_train(X_train=X_train0, y_train=y_train0, X_test=X_test0, y_test=y_test0)

Train on 1000 samples, validate on 944 samples
Epoch 1/30
1000/1000 [==============================] - 252s 252ms/step - loss: 0.6926 - acc: 0.5490 - val_loss: 0.6913 - val_acc: 0.6250
Epoch 2/30
1000/1000 [==============================] - 253s 253ms/step - loss: 0.6824 - acc: 0.6070 - val_loss: 0.6781 - val_acc: 0.6250
Epoch 3/30
1000/1000 [==============================] - 257s 257ms/step - loss: 0.6758 - acc: 0.6070 - val_loss: 0.6975 - val_acc: 0.6250
Epoch 4/30
1000/1000 [==============================] - 253s 253ms/step - loss: 0.6676 - acc: 0.6070 - val_loss: 0.6696 - val_acc: 0.6250
Epoch 5/30
1000/1000 [==============================] - 254s 254ms/step - loss: 0.6721 - acc: 0.6070 - val_loss: 0.6624 - val_acc: 0.6250
Epoch 6/30
1000/1000 [==============================] - 255s 255ms/step - loss: 0.6744 - acc: 0.6070 - val_loss: 0.6632 - val_acc: 0.6250
Epoch 7/30
1000/1000 [==============================] - 254s 254ms/step - loss: 0.6752 - acc: 0.6070 - val_loss: 0.6732 - val

KeyboardInterrupt: 

In [18]:
classifier = LeNetMnistClassifier()
X_train1, y_train1, X_test1, y_test1 = classifier._preprocess(X_train0, y_train0, X_test0, y_test0)

In [14]:
classifier2 = LeNetMnistClassifier(epochs=20, activation_function='elu')
classifier2.preprocess_and_train(X_train=X_train0, y_train=y_train0, X_test=X_test0, y_test=y_test0)

Train on 1000 samples, validate on 944 samples
Epoch 1/20
1000/1000 [==============================] - 259s 259ms/step - loss: 0.6714 - acc: 0.5940 - val_loss: 0.6771 - val_acc: 0.6250
Epoch 2/20
1000/1000 [==============================] - 259s 259ms/step - loss: 0.6812 - acc: 0.6060 - val_loss: 0.6616 - val_acc: 0.6250
Epoch 3/20
1000/1000 [==============================] - 260s 260ms/step - loss: 0.6686 - acc: 0.6090 - val_loss: 0.6585 - val_acc: 0.6250
Epoch 4/20


KeyboardInterrupt: 

In [21]:
base_model= keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
features = base_model.predict(X_test1)
features = features.reshape(images.shape[0], features.shape[1]*features.shape[2]*features.shape[3])

loss_function='categorical_crossentropy'
optimizer=keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
base_model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
        
fit_output = base_model.fit(X_train1,
                        y_train1,
                        batch_size=128,
                        epochs=20,
                        verbose=1,
                        validation_data=(X_test1, y_test1))


ValueError: The input must have 3 channels; got `input_shape=(224, 224, 1)`